In [1]:
from jetbot import Camera, bgr8_to_jpeg

In [4]:
image = widgets.Image(format='jpeg', width=300, height=300)

In [5]:
camera = Camera.instance(capture_height=720, capture_width=1080, fps=30)


In [6]:
traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

In [7]:
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [8]:
from IPython.display import display
import traitlets
import ipywidgets.widgets as widgets
import sys
sys.path.append('/opt/nvidia/jetson-gpio/lib/python/')
sys.path.append('/opt/nvidia/jetson-gpio/lib/python/Jetson/GPIO')
from adafruit_servokit import ServoKit

In [9]:
class Racecar(traitlets.HasTraits):
    
    throttle = traitlets.Float(default_value=0.0)
    steering = traitlets.Float(default_value=0.0)
    
    def __init__(self):
        self._kit = ServoKit(channels=16)
        self._steering_servo = self._kit.continuous_servo[0]
        self._throttle_motor = self._kit.continuous_servo[1]
        
    @traitlets.observe('throttle')
    def _on_throttle(self, change):
        self._throttle_motor.throttle = -change['new']
        
    @traitlets.validate('throttle')
    def _validate_throttle(self, proposal):
        if proposal['value'] > 1.0:
            return 1.0
        if proposal['value'] < -1.0:
            return -1.0
        return proposal['value']
        
    @traitlets.validate('steering')
    def _validate_steering(self, proposal):
        if proposal['value'] > 1.0:
            return 1.0
        if proposal['value'] < -1.0:
            return -1.0
        return proposal['value']
        
    @traitlets.observe('steering')
    def _on_steering(self, change):
        self._steering_servo.throttle = change['new']

In [10]:
car = Racecar()

In [14]:
controller = widgets.Controller(index=0)

display(controller)

Controller()

In [12]:
steering_gain = widgets.FloatSlider(min=0.0, max=1.0, value=1.0, description='steering gain')
steering_trim = widgets.FloatSlider(min=-0.2, max=0.2, value=0.0, step=0.001, description='steering trim')
throttle_gain = widgets.FloatSlider(min=0.0, max=1.0, value=0.3, description='throttle gain')

display(steering_gain, steering_trim, throttle_gain)

FloatSlider(value=1.0, description='steering gain', max=1.0)

FloatSlider(value=0.0, description='steering trim', max=0.2, min=-0.2, step=0.001)

FloatSlider(value=0.3, description='throttle gain', max=1.0)

In [15]:
traitlets.dlink((controller.axes[2], 'value'), (car, 'steering'), transform=lambda x: x * steering_gain.value + steering_trim.value)

In [16]:
traitlets.dlink((controller.axes[1], 'value'), (car, 'throttle'), transform=lambda x: -x * throttle_gain.value)

In [65]:
controller.axes[2].unobserve_all()